# Geração de instâncias

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from udgp import M1, M2, M4, M5, Instance


In [2]:
env = gp.Env()


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br


In [3]:
coords = np.array(
    [
        [0.2730989500, 1.1469629952, -0.3319578813],
        [-0.4728837298, -0.6223685080, 0.7664213265],
        [-0.9666537615, -0.2393056630, -0.1698094248],
        [0.6209419539, -0.3628130566, 0.7094425990],
        [0.8035441992, 0.1648033307, -0.2639206823],
        [-0.1784380914, 0.2412141513, -0.8077599510],
        [0.0639788373, -0.6647479592, -0.2089132333],
        [-0.1435883576, 0.3362547097, 0.3064972473],
    ]
)

In [4]:
N = 6
# instance = Instance.random(N, freq=True)
instance = Instance.from_coords(coords.round(3), freq=True)

instance.view_input()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
instance.dist


array([1.11, 1.12, 1.13, 1.14, 1.82, 1.83, 1.87, 2.21], dtype=float16)

In [7]:
instance.dist = instance.dist.round(2)


In [7]:
instance.reset()
m = M4(instance, env=env)
m.setParam("TimeLimit", 10 * 60)
m.optimize()

m.instance.view()

In [8]:
# previous_a = []

# TODO: evitar que o cara encontre a mesma solução errada várias vezes.

# while not instance.is_solved():
while instance.fixed_n < instance.n:
    print("reset")
    instance.reset_random_core(5)

    # m = M4(instance, n=4, previous_a=previous_a, env=env)
    # # m.Params.TimeLimit = 120
    # m.optimize()

    # previous_a.append(list(m.a_ijk_values()))
    # print(list(m.a_ijk_values()))

    while instance.fixed_n < instance.n:
        print("step")
        m = M4(instance, n=1, env=env)
        m.setParam("TimeLimit", 60)
        m.optimize()

        if m.Status is not GRB.OPTIMAL:
            break

instance.view()

reset
step


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol